# Preparação, Treinamento e Prova de Conceito

**Importação**

In [1]:
#importacao de dados
import pandas as pd


df=pd.read_csv('udemy_output_All_IT__Software_p1_p626.csv')

**Preparação do dados**

In [22]:
#Preparacao dos dados
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


# Preenchendo valores nulos no título
df['title'] = df['title'].fillna('')

# Ajusta as stop word para o contexto da base de cursos
stop_words = stopwords.words('english') + ['course', 'learn', 'training',
                                           'online', 'award', 'success']

# Transformando os títulos em vetores TF-IDF
vectorizer = TfidfVectorizer(stop_words=stop_words)
tfidf_matrix = vectorizer.fit_transform(df['title'])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Treinamento**

In [23]:
#Técnica de recomendação
from sklearn.metrics.pairwise import cosine_similarity


# Utilizando similaridade do cosseno entre os vetores dos títulos
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

**Prova de conceito**

In [26]:
import random

def exibir_recomendacoes(course_id, top_n=3):
    curso_alvo = df[df['id'] == course_id]['title'].values[0]
    print(f"\n Curso selecionado: {curso_alvo}\n")

    # Recomendação
    idx = df[df['id'] == course_id].index[0]
    # retorna um vetor de similaridade entre o curso-alvo e todos os outros cursos.
    sim_scores = list(enumerate(cosine_sim[idx]))
    # Ordena os cursos com base na similaridade
    # Ignora o primeiro resultado (posição 0), porque ele será o curso com similaridade 1.0 com ele mesmo.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    #Extrai só os índices dos cursos recomendados.
    recommended_indices = [i[0] for i in sim_scores]

    print("Cursos recomendados:")
    for i in recommended_indices:
        print(f"  - {df.iloc[i]['title']}")

ids_disponiveis = df['id'].tolist()
ids_para_testar = random.choices(ids_disponiveis, k=5)

# Rodando 5 provas de conceito
for i, cid in enumerate(ids_para_testar[:5], start=1):
    print(f"\n Teste {i}/5")
    exibir_recomendacoes(cid, top_n=3)


 Teste 1/5

 Curso selecionado: Selenium WebDriver Bootcamp - Beginner to Pro

Cursos recomendados:
  - Selenium WebDriver Masterclass with C#
  - Selenium Webdriver in Java
  - Selenium Webdriver for SDET in Java - Bootcamp

 Teste 2/5

 Curso selecionado: Practice Exams LPIC-2 Engineer (201-450 & 202-450) | QA PDF

Cursos recomendados:
  - LPIC-2 Linux Engineer Practice Exam (201-450 & 202-450) 2020
  - LPIC-2 - Linux Engineer Exam 201-450 - June/2020 Q/A UPDATED
  - CISSP Practice Exam (All Domains) 450 Questions

 Teste 3/5

 Curso selecionado: Recruiter Training - Hire Top Employees in 30 days

Cursos recomendados:
  - How to Launch Your Business in 30 Days
  - IT/NON-IT Recruiter Training to become a Recruiter (Junior)
  - Online Business: 30 Days Challenge

 Teste 4/5

 Curso selecionado: Business Modelling Masterclass- Business Meets Funding

Cursos recomendados:
  - Absolute Entrepreneurship Part 2: Funding Your Business
  - BPMN 2.0 Business Process Modelling for Business An

**Métricas**

In [37]:
from sklearn.metrics import ndcg_score
import numpy as np

# -------------------------------
# MÉTRICA 1: Precision@K
# -------------------------------
def precision_at_k(course_id, top_n=3, threshold=0.3):
    idx = df[df['id'] == course_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    relevantes = [1 if score[1] >= threshold else 0 for score in sim_scores]
    precision = sum(relevantes) / top_n
    return precision

# -------------------------------
# MÉTRICA 2: Recall@K
# -------------------------------
def recall_at_k(course_id, top_n=3, threshold=0.3):
    idx = df[df['id'] == course_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    relevantes_totais = sum(1 for score in sim_scores if score[1] >= threshold and score[0] != idx)

    if relevantes_totais == 0:
        return 0.0

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    relevantes_encontrados = sum(1 if score[1] >= threshold else 0 for score in sim_scores)
    recall = relevantes_encontrados / relevantes_totais
    return recall


# -------------------------------
# MÉTRICA 3: MRR (Mean Reciprocal Rank)
# -------------------------------
def mrr(course_id, threshold=0.3):
    idx = df[df['id'] == course_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    for rank, (i, score) in enumerate(sim_scores[1:], start=1):
        if score >= threshold:
            return 1 / rank
    return 0.0

# -------------------------------
# MÉTRICA 4: NDCG@K (Normalized Discounted Cumulative Gain)
# -------------------------------
def ndcg_at_k(course_id, top_n=3):
    idx = df[df['id'] == course_id].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]

    # Gera relevância simulada com base na própria similaridade
    y_true = [score[1] for score in sim_scores]
    y_score = [score[1] for score in sim_scores]

    return ndcg_score([y_true], [y_score])



In [38]:
 #Rodar o cálculo das métricas para todos os cursos da base simulada

todos_ids = df['id'].tolist()

# Reavaliar para todos os cursos
all_precisions = []
all_recalls = []
all_mrrs = []
all_ndcgs = []

for cid in todos_ids:
    all_precisions.append(precision_at_k(cid))
    all_recalls.append(recall_at_k(cid))
    all_mrrs.append(mrr(cid))
    all_ndcgs.append(ndcg_at_k(cid))

# Resultados médios em toda a base
metrics_summary_full = {
    "Precision@3": round(np.mean(all_precisions), 2),
    "Recall@3": round(np.mean(all_recalls), 2),
    "MRR": round(np.mean(all_mrrs), 2),
    "NDCG@3": round(np.mean(all_ndcgs), 2)
}

metrics_summary_full

KeyboardInterrupt: 